# Experiment on the encoding and imputing technics for the Nam network, with and without Z ! 

In [1]:
run init.ipynb

Done.


**`NAN` Second set of experiments deciding wheether `using indicator variable is useful`**

In [5]:
"""  Parameters of the experiment """
approach = 'xgboost'

range_dataset_name = ['circles', 'moons', 'blobs']
range_ratio_of_missing_values = [0., 0.1, 0.2, 0.3]
range_ratio_missing_per_class = [[0,0.25], [0.1, 0.3]]
range_imbalance_ratio = [0.5, 0.25, 0.1]
range_missingness_pattern = [1, 3, 4, 5]

range_missing_data_handling = ['without', 'encoding'] #TODO THIS IS WHERE WE SHOULD DO IMUTING BASED ON THE  OTHER EXPERIMENT
imputation_method = 'without'
range_use_missing_indicator_variables = [True, False]

In [3]:
df = create_df(folder_names=['experiments_24_june_duke', 'experiments_24_june_duke', 'experiments_solidays', 'experiement_28_june', 'experiement_28_june_duke'])
df

,dataset_name,experiment_number,approach,missing_data_handling,imputation_method,use_missing_indicator_variables,num_samples,imbalance_ratio,missingness_pattern,missingness_mechanism,ratio_of_missing_values,missing_X1,missing_X2,missing_first_quarter,ratio_missing_per_class_0,ratio_missing_per_class_1,auc,Accuracy,F1,MCC,Sensitivity,Specificity,Precision,PPV,NPV,FNR,FDR,FOR,resolution,bandwidth,estimation_time_0,estimation_time_1
0,circles,47,nam,encoding,without,True,5000,0.25,5,MNAR,NaN,True,True,True,0.0,0.2376,0.9902,0.8109,0.3812,0.4338,0.2360,1.0000,1.0000,1.0000,0.7991,0.7640,0.0000,0.2009,20,0.2,NaN,NaN
1,circles,1,nam,encoding,without,True,5000,0.10,1,MCAR,0.0,True,False,None,None,None,0.9868,0.9010,0.0000,NaN,0.0000,1.0000,NaN,NaN,0.9010,1.0000,NaN,0.0990,20,0.2,NaN,NaN
2,circles,45,nam,encoding,without,True,5000,0.25,4,MNAR,None,True,True,False,0,0.25,0.9920,0.8584,0.5996,0.6004,0.4286,1.0000,1.0000,1.0000,0.8417,0.5714,0.0000,0.1583,20,0.2,NaN,NaN
3,circles,13,nam,encoding,without,True,5000,0.10,4,MNAR,None,True,True,False,0,0.25,0.9909,0.9420,0.5887,0.6262,0.4191,1.0000,1.0000,1.0000,0.9394,0.5809,0.0000,0.0606,20,0.2,NaN,NaN
4,circles,31,nam,encoding,without,True,5000,0.10,5,MNAR,None,True,True,True,0.0,0.208,0.9872,0.9253,0.3677,0.4561,0.2258,1.0000,1.0000,1.0000,0.9234,0.7742,0.0000,0.0766,20,0.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,moons,56,nam,encoding,without,True,5000,0.25,2,MAR,0.3,True,True,True,None,None,0.9480,0.9295,0.8350,0.8060,0.7283,0.9952,0.9801,0.9801,0.9184,0.2717,0.0199,0.0816,20,0.2,NaN,NaN
255,moons,62,nam,encoding,without,True,5000,0.25,4,MNAR,None,True,True,False,0.1,0.3,0.9630,0.8937,0.7455,0.7126,0.6111,0.9917,0.9631,0.9631,0.8808,0.3889,0.0369,0.1192,20,0.2,NaN,NaN
256,moons,60,nam,encoding,without,True,5000,0.25,3,MAR,0.3,True,False,True,None,None,0.9725,0.9354,0.8561,0.8271,0.7578,0.9959,0.9842,0.9842,0.9237,0.2422,0.0158,0.0763,20,0.2,NaN,NaN
257,moons,10,nam,encoding,without,True,5000,0.10,3,MAR,0.1,True,False,True,None,None,0.9797,0.9681,0.8245,0.8203,0.7147,0.9983,0.9778,0.9778,0.9671,0.2853,0.0222,0.0329,20,0.2,NaN,NaN


In [13]:
df = create_df(folder_names=['experiments_24_june_duke'])
df

,dataset_name,experiment_number,approach,missing_data_handling,imputation_method,use_missing_indicator_variables,num_samples,imbalance_ratio,missingness_pattern,missingness_mechanism,ratio_of_missing_values,missing_X1,missing_X2,missing_first_quarter,ratio_missing_per_class_0,ratio_missing_per_class_1,auc,Accuracy,F1,MCC,Sensitivity,Specificity,Precision,PPV,NPV,FNR,FDR,FOR,resolution,bandwidth,estimation_time_0,estimation_time_1
0,circles,47,nam,encoding,without,True,5000,0.25,5,MNAR,NaN,True,True,True,0.0,0.2376,0.9902,0.8109,0.3812,0.4338,0.2360,1.0000,1.0000,1.0000,0.7991,0.7640,0.0000,0.2009,20,0.2,NaN,NaN
1,circles,1,nam,encoding,without,True,5000,0.10,1,MCAR,0.0,True,False,None,None,None,0.9868,0.9010,0.0000,NaN,0.0000,1.0000,NaN,NaN,0.9010,1.0000,NaN,0.0990,20,0.2,NaN,NaN
2,circles,45,nam,encoding,without,True,5000,0.25,4,MNAR,None,True,True,False,0,0.25,0.9920,0.8584,0.5996,0.6004,0.4286,1.0000,1.0000,1.0000,0.8417,0.5714,0.0000,0.1583,20,0.2,NaN,NaN
3,circles,13,nam,encoding,without,True,5000,0.10,4,MNAR,None,True,True,False,0,0.25,0.9909,0.9420,0.5887,0.6262,0.4191,1.0000,1.0000,1.0000,0.9394,0.5809,0.0000,0.0606,20,0.2,NaN,NaN
4,circles,31,nam,encoding,without,True,5000,0.10,5,MNAR,None,True,True,True,0.0,0.208,0.9872,0.9253,0.3677,0.4561,0.2258,1.0000,1.0000,1.0000,0.9234,0.7742,0.0000,0.0766,20,0.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,moons,56,nam,encoding,without,True,5000,0.25,2,MAR,0.3,True,True,True,None,None,0.9480,0.9295,0.8350,0.8060,0.7283,0.9952,0.9801,0.9801,0.9184,0.2717,0.0199,0.0816,20,0.2,NaN,NaN
255,moons,62,nam,encoding,without,True,5000,0.25,4,MNAR,None,True,True,False,0.1,0.3,0.9630,0.8937,0.7455,0.7126,0.6111,0.9917,0.9631,0.9631,0.8808,0.3889,0.0369,0.1192,20,0.2,NaN,NaN
256,moons,60,nam,encoding,without,True,5000,0.25,3,MAR,0.3,True,False,True,None,None,0.9725,0.9354,0.8561,0.8271,0.7578,0.9959,0.9842,0.9842,0.9237,0.2422,0.0158,0.0763,20,0.2,NaN,NaN
257,moons,10,nam,encoding,without,True,5000,0.10,3,MAR,0.1,True,False,True,None,None,0.9797,0.9681,0.8245,0.8203,0.7147,0.9983,0.9778,0.9778,0.9671,0.2853,0.0222,0.0329,20,0.2,NaN,NaN


In [8]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):




                        if check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                             missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class=ratio_missing_per_class):


                            print("Experiment already done.")
                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1):
                            print("Not doing this.")
                            continue
                            
                            
                            
                        print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                        missing_data_handling,
                                                                                                                                                          imputation_method, 
                                                                                                                                                          imbalance_ratio,
                                                                                                                                                          missingness_pattern,
                                                                                                                                                          ratio_of_missing_values, 
                                                                                                                                                              ratio_missing_per_class))

                        try:
                            start_time = time()

                            dataset = DatasetGenerator(dataset_name=dataset_name, 
                                                    num_samples=num_samples, 
                                                    imbalance_ratio=imbalance_ratio, 
                                                    missing_data_handling=missing_data_handling,
                                                    imputation_method=imputation_method,
                                                    verbosity=verbosity)

                            # Creation of the missingness
                            dataset.generate_missing_coordinates(missingness_pattern=missingness_pattern, 
                                                                 ratio_of_missing_values=ratio_of_missing_values, 
                                                                 ratio_missing_per_class=ratio_missing_per_class)

                            dataset.split_test_train()

                            # Create the experiments
                            exp = Experiments(dataset_name, 
                                              dataset=dataset, 
                                              purpose='classification', 
                                              approach = approach, 
                                              previous_experiment=None, 
                                              save_experiment=save, 
                                              verbosity=verbosity, 
                                              debug=False, 
                                              proportion_train=PROPORTION_TRAIN, 
                                              resolution=RESOLUTION, 
                                              bandwidth=BANDWIDTH,
                                              random_state=RANDOM_STATE)


                            # Estimate the distributions 
                            exp.fit()

                            # Estimate the distributions 
                            exp.predict()

                            # Plot results
                            #exp.plot()


                            end_time = time()

                            hours, rest = divmod(end_time - start_time, 3600)
                            minutes, seconds = divmod(rest, 60)
                            print("Done ({}h {}m and {:.2f}s).\n".format(int(hours), int(minutes), seconds))
                        except:
                            print("/!\. Error!\n")


circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.0 ratio_per_class: None
Doing experiment 1!
Done (0h 0m and 1.24s).

circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.1 ratio_per_class: None
Doing experiment 2!
Done (0h 0m and 1.58s).

circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.2 ratio_per_class: None
Doing experiment 3!
Done (0h 0m and 1.51s).

circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.3 ratio_per_class: None
Doing experiment 4!
Done (0h 0m and 1.40s).

circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 2 ratio_missing:0.0 ratio_per_class: None
Doing experiment 5!
Done (0h 0m and 1.16s).

circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 2 ratio_missi